In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import collections

In [ ]:
# get callback in XXX node
def cb(node):    
    path = '/home/rei/reigit/avpdemo.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    #node handle
    S = l.index('Nodes:\n')
    E = l.index('Publishers:\n')
    node = [line for line in l[S:E] if node in line]
    nodename= node[0].split()[4]
    nodeh= node[0].split()[0]

    S = l.index('Subscriptions:\n')
    E = l.index('Subscription objects:\n')
    subt = [line for line in l[S:E] if nodeh in line]
    stopic=[]
    for i in subt:
        stopic.append('/'+i.split(' /')[1].split()[0])
    print('sub:',stopic)
    
    S = l.index('Publishers:\n')
    E = l.index('Subscriptions:\n')
    pubt = [line for line in l[S:E] if nodeh in line]
    ptopic=[]
    for i in pubt:
        ptopic.append('/'+i.split(' /')[1].split()[0])
    print('pub:',ptopic)
    
    S = l.index('Services:\n')
    E = l.index('Clients:\n')
    sert = [line for line in l[S:E] if nodeh in line]
    sname=[]
    for i in sert:
        sname.append('/'+i.split(' /')[1].split()[0])
    print('service:',sname)
    
    S = l.index('Clients:\n')
    E = l.index('Timers:\n')
    clit = [line for line in l[S:E] if nodeh in line]
    csname=[]
    for i in clit:
        csname.append('/'+i.split(' /')[1].split()[0])
    print('client:',csname)

In [ ]:
# get callback duration
def cbd(node,topic):
    path = '/home/rei/reigit/avpdemo.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    
    #node handle
    S = l.index('Nodes:\n')
    E = l.index('Publishers:\n')
    node = [line for line in l[S:E] if node in line]
    nodename= node[0].split()[4]
    nodeh= node[0].split()[0]

    S = l.index('Subscriptions:\n')
    E = l.index('Subscription objects:\n')
    subt = [line for line in l[S:E] if nodeh in line]
    st=[line for line in subt if topic in line]
    subscription_handle= st[0].split()[0]
    
    #subscription_handle -> reference
    S = l.index('Subscription objects:\n')
    E = l.index('Services:\n')   
    refer=[]
    ref = [line for line in l[S:E] if subscription_handle in line]
    refer.append(''.join(ref))
    reference= refer[0].split()[0]
    
    #reference -> callback_object
    S = l.index('Callback objects:\n')
    E = l.index('Callback symbols:\n')   
    cboj=[]
    cb= [line for line in l[S:E] if reference in line]
    cboj.append(''.join(cb))
    callback_object=cboj[0].split()[2]

    t=[]
    with open(path) as f:
        l = f.readlines()
    S = l.index('Callback instances:\n')
    E = l.index('Lifecycle state machines:\n')  
    l_XXX = [line for line in l[S:E] if callback_object in line]

    # S = l.index('Callback symbols:\n')
    # E = l.index('Callback instances:\n')  
    # cb_name = [line for line in l[S:E] if callback_object in line]
    # cb_name = ''.join(cb_name)
    # cb_name = cb_name.split()[3]

    cbd_ns = []
    for i in l_XXX:
        cbd_ns.append(int(i.split()[3])) 
    cbd_us = []
    for i in cbd_ns:
        cbd_us.append(i/1000)

    return cbd_us,nodename

In [ ]:
# calculate the voxel_grid_cloud_node com latency
def ps_com(node,topic):
    path = '/home/rei/reigit/avpdemo.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    
    #node handle
    S = l.index('Nodes:\n')
    E = l.index('Publishers:\n')
    node = [line for line in l[S:E] if node in line]
    nodename= node[0].split()[4]
    nodeh= node[0].split()[0]

    S = l.index('Subscriptions:\n')
    E = l.index('Subscription objects:\n')
    subt = [line for line in l[S:E] if nodeh in line]
    st=[line for line in subt if topic in line]
    subscription_handle= st[0].split()[0]
    
    #subscription_handle -> reference
    S = l.index('Subscription objects:\n')
    E = l.index('Services:\n')   
    refer=[]
    ref = [line for line in l[S:E] if subscription_handle in line]
    refer.append(''.join(ref))
    reference= refer[0].split()[0]
    
    #reference -> callback_object
    S = l.index('Callback objects:\n')
    E = l.index('Callback symbols:\n')   
    cboj=[]
    cb= [line for line in l[S:E] if reference in line]
    cboj.append(''.join(cb))
    callback_object=cboj[0].split()[2]
    
    cbh = str(hex(int(callback_object)))
    cbhandle = cbh.upper().replace('X', 'x')

    path = path = '/home/rei/reigit/rclcpp_subscribe.txt'
    f = open(path)
    f.close()
    with open(path) as f:
        l = f.readlines()
    
    com_latency_ns = []
    for i in l:
        #print(i)
        callback = i.split(",")[4].split("= ")[1]
        #print(callback)
        if (callback == cbhandle):
            source = i.split("source_stamp = ")[1].split(",")[0]
            receive = i.split("received_stamp = ")[1].split(",")[0].split()[0]
            com = int(receive) - int(source)
            com_latency_ns.append(com)

    com_latency_us = []
    for i in com_latency_ns:
        com_latency_us.append(i/1000)
    return com_latency_us,nodename+'_com'

In [ ]:
# node latency + com latency
def node_com(cbduration,comlatency):
    node_com_latency = [int(cbduration[i])+int(comlatency[i]) for i in range(min(len(cbduration),len(comlatency)))]
    return node_com_latency